In [ ]:
import pandas as pd
from datetime import datetime, timedelta

start_date = datetime(2020,1,1)
end_date = datetime(2020,12,31)

year_list = []
month_list = []
day_list = []
date_list = []
yr_mth_list = []

current_date = start_date
while current_date <= end_date:
  year_list.append(current_date.year)
  month_list.append(current_date.month)
  day_list.append(current_date.day)
  date_list.append(current_date.strftime('%Y-%m-%d'))
  yr_mth_list.append(current_date.strftime('%Y%m'))
  current_date += timedelta(days=1)

date_df = pd.DataFrame({
  'Year': year_list,
  'Month': month_list,
  'Day': day_list,
  'Date': date_list,
  'YrMth': yr_mth_list,
})

In [ ]:
date_df

In [ ]:
date_df['YrMth'].unique()

In [97]:
import json
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as BS
import os
import glob
from tqdm import tqdm
import time

In [ ]:
date_list = []
data_list = []
for i in tqdm(date_df['YrMth'].unique()):
  start_row = 1
  end_row = 1000

  
  while True:
    api_key = '72466d45736c6f7633357649627049'
    url = f'http://openapi.seoul.go.kr:8088/{api_key}/json/tbCycleRentUseMonthInfo/{start_row}/{end_row}/{i}'
    r = requests.get(url)
    data = r.json()
    try:
      data_list.append(pd.DataFrame(r.json()['cycleRentUseMonthInfo']['row']))
    except:
      pass
    if 'INFO-200' in r.text: 
      break
    start_row += 1000
    end_row += 1000
    print(url)
  date_list.append(data_list)
  print(url)
print(date_list)

In [ ]:
date_list

In [ ]:
date_list

In [ ]:
CycleRentUseMonth_2020_df = pd.concat(data_list)

In [ ]:
CycleRentUseMonth_2020_df

In [94]:
import pickle

with open('./sourcefile/data/CycleRentUseMonth_2020_raw.pkl', 'wb') as f:
  pickle.dump(CycleRentUseMonth_2020_df,f)

In [95]:
df = pickle.load(open('./sourcefile/data/CycleRentUseMonth_2020_raw.pkl','rb'))

In [98]:
df

,RENT_NM,RENT_TYPE,STATION_NO,STATION_NAME,GENDER_CD,AGE_TYPE,USE_CNT,EXER_AMT,CARBON_AMT,MOVE_METER,MOVE_TIME,START_INDEX,END_INDEX,RNUM
0,202001,정기권,00102,102. 망원역 1번출구 앞,,~10대,1,32.58,0.28,1210.00,6,0,0,712
1,202001,정기권,00103,103. 망원역 2번출구 앞,,~10대,16,550.77,5.68,24490.00,172,0,0,781
2,202001,정기권,00104,104. 합정역 1번출구 앞,,~10대,2,144.00,1.28,5550.00,49,0,0,866
3,202001,정기권,00105,105. 합정역 5번출구 앞,,~10대,5,582.25,5.25,22620.00,97,0,0,933
4,202001,정기권,00106,106. 합정역 7번출구 앞,,~10대,4,160.72,1.51,6540.00,67,0,0,986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,202012,정기권,03548,3548.뚝섬역 6번출구,M,70대이상,3,67.40,0.66,2836.59,67,0,0,104183
244,202012,정기권,03600,3600.사근빗물펌프장 건너편,M,기타,5,0.00,0.00,0.00,95,0,0,104339
245,202012,정기권,03787,3787. 가양나들목,M,70대이상,1,0.00,0.00,0.00,86,0,0,104190
246,202012,정기권,03902,3902. 항동지구5단지 501동 앞,M,70대이상,7,0.00,0.00,0.00,577,0,0,104188


In [99]:
df['RENT_NM'].dtype

dtype('O')

In [100]:
df = df.astype({
  'USE_CNT' : 'int',
  'STATION_NO': 'int'
})

In [101]:
df['USE_CNT'].dtype

dtype('int64')

In [102]:
df1 = df.groupby(['RENT_NM', 'STATION_NO'])['USE_CNT'].sum()

In [103]:
df1 = df1.reset_index()

In [104]:
df1 = df1.rename(columns={
  'RENT_NM': 'Rent_Date',
  'STATION_NO': 'Rent_Number',
  'USE_CNT': 'Rent_Count',
})

In [105]:
df1

,Rent_Date,Rent_Number,Rent_Count
0,202001,101,305
1,202001,102,1324
2,202001,103,758
3,202001,104,903
4,202001,105,501
...,...,...,...
22910,202012,4776,5
22911,202012,4778,17
22912,202012,4784,17
22913,202012,4785,6


In [106]:
df1['Rent_Date'] = df1['Rent_Date'].apply(lambda x : str(x)[:4] + '-' + str(x)[4:])

In [107]:
df1

,Rent_Date,Rent_Number,Rent_Count
0,2020-01,101,305
1,2020-01,102,1324
2,2020-01,103,758
3,2020-01,104,903
4,2020-01,105,501
...,...,...,...
22910,2020-12,4776,5
22911,2020-12,4778,17
22912,2020-12,4784,17
22913,2020-12,4785,6


In [109]:
with open('./sourcefile/data/CycleRentUseMonth_2020.pkl', 'wb') as f:
  pickle.dump(df1,f)

In [111]:
pickle.load(open('./sourcefile/data/CycleRentUseMonth_2020.pkl','rb'))

,Rent_Date,Rent_Number,Rent_Count
0,2020-01,101,305
1,2020-01,102,1324
2,2020-01,103,758
3,2020-01,104,903
4,2020-01,105,501
...,...,...,...
22910,2020-12,4776,5
22911,2020-12,4778,17
22912,2020-12,4784,17
22913,2020-12,4785,6


In [113]:
pickle.load(open('./sourcefile/data/CycleRentUseMonth_2020_raw.pkl','rb'))

,RENT_NM,RENT_TYPE,STATION_NO,STATION_NAME,GENDER_CD,AGE_TYPE,USE_CNT,EXER_AMT,CARBON_AMT,MOVE_METER,MOVE_TIME,START_INDEX,END_INDEX,RNUM
0,202001,정기권,00102,102. 망원역 1번출구 앞,,~10대,1,32.58,0.28,1210.00,6,0,0,712
1,202001,정기권,00103,103. 망원역 2번출구 앞,,~10대,16,550.77,5.68,24490.00,172,0,0,781
2,202001,정기권,00104,104. 합정역 1번출구 앞,,~10대,2,144.00,1.28,5550.00,49,0,0,866
3,202001,정기권,00105,105. 합정역 5번출구 앞,,~10대,5,582.25,5.25,22620.00,97,0,0,933
4,202001,정기권,00106,106. 합정역 7번출구 앞,,~10대,4,160.72,1.51,6540.00,67,0,0,986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,202012,정기권,03548,3548.뚝섬역 6번출구,M,70대이상,3,67.40,0.66,2836.59,67,0,0,104183
244,202012,정기권,03600,3600.사근빗물펌프장 건너편,M,기타,5,0.00,0.00,0.00,95,0,0,104339
245,202012,정기권,03787,3787. 가양나들목,M,70대이상,1,0.00,0.00,0.00,86,0,0,104190
246,202012,정기권,03902,3902. 항동지구5단지 501동 앞,M,70대이상,7,0.00,0.00,0.00,577,0,0,104188
